In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("~/SFUSD/Data/Cleaned/drop_optout_1819.csv", low_memory=False)
df = df.loc[df.grade == "KG"]
df = df.loc[df["sibling"] != "[]"]
# df = df.dropna(subset=["sibling"])
# df.loc[:, "sibling"] = df.sibling.apply(lambda x: eval(x)[0])

In [3]:
pre_merge_len = len(df)
real_match = pd.read_csv("~/Documents/sfusd/local_runs/assignments/Assignment_real_match.csv")
df = df.merge(real_match[["studentno", "programcodes"]], how="left", on="studentno")
df.rename(columns={"programcodes": "program_id"}, inplace=True)
assert len(df) == pre_merge_len

In [4]:
# assigned = df.groupby("sibling").count()[["studentno"]]
# assigned.loc[:, "program_id"] = [f"{x}-GE-KG" for x in assigned.index]
assigned = df[["studentno", "program_id"]].groupby("program_id", as_index=False).count()

In [5]:
assigned

,program_id,studentno
0,413-GE-KG,26
1,420-GE-KG,13
2,420-SE-KG,12
3,420-SN-KG,3
4,435-GE-KG,28
...,...,...
120,872-CB-KG,9
121,872-GE-KG,3
122,876-CE-KG,10
123,876-CN-KG,10


In [6]:
df[["studentno", "sibling"]]

,studentno,sibling
0,10210006,[729]
1,10210058,[872]
2,10210137,[509]
3,10210149,[509]
4,10210160,[676]
...,...,...
1296,888009715,[449]
1297,888020901,[509]
1298,888021277,[656]
1299,888021301,[525]


In [7]:
unassigned_sibs = df.loc[df.program_id.isna()]
unassigned_sibs

,studentno,r1_ranked_idschool,r1_listed_ranks,r1_programs,grade,r1_randomnumber,r1_cohortstring,bayview_to_all_ms,brown_ms_to_hs,bayview_to_brown_ms,...,FRL Score,sibling,currentlpsibling,currentlp,aaprek,aa,median_hh_income,lowell_ranked,sota_ranked,program_id
170,10216309,"[485, 509, 644, 676]","[1, 2, 3, 4]","['CT', 'CN', 'GE', 'GE']",KG,"[0.392533209163712, 0.197585107090507, 0.47592...","['', '', '', '']",0.0,0.0,0.0,...,0.538793,[644],[],[],[],[],86518.0,0,0,NaN
381,10218150,"[746, 485, 838, 876, 676, 796, 750, 862, 539]","[1, 2, 3, 4, 6, 7, 8, 9, 10]","['ME', 'CE', 'ME', 'CE', 'GE', 'GE', 'GE', 'GE...",KG,"[0.0602299019347038, 0.764347973158287, 0.1649...","['S;', '', '', '', '', '', '', '', 'AA;']",0.0,0.0,0.0,...,0.428942,[746],[],[],[],[539],122679.0,0,0,NaN
474,10219027,"[746, 539, 478, 478]","[1, 2, 3, 4]","['MN', 'GE', 'JE', 'GE']",KG,"[0.223325791336533, 0.271217740082699, 0.94392...","['', 'S;', '', '']",0.0,0.0,0.0,...,0.107759,[539],[],[],[],[],155985.0,0,0,NaN
1155,888022694,[507],[1],['GE'],KG,[0.332621089866951],['CT1;S;'],0.0,0.0,0.0,...,0.789762,[507],[],[],[],[],NaN,0,0,NaN
1200,10242356,NaN,NaN,NaN,KG,NaN,NaN,0.0,0.0,0.0,...,0.706705,[814],[],[],[],[],160577.0,0,0,NaN
1205,10242390,NaN,NaN,NaN,KG,NaN,NaN,0.0,0.0,0.0,...,0.538793,[714],[],[],[],[],159350.0,0,0,NaN
1228,10242581,NaN,NaN,NaN,KG,NaN,NaN,0.0,0.0,0.0,...,0.032654,[420],[],[],[],[],154333.0,0,0,NaN
1242,10242778,NaN,NaN,NaN,KG,NaN,NaN,0.0,0.0,0.0,...,0.743574,[760],[],[],[],[],73600.0,0,0,NaN
1243,10242797,NaN,NaN,NaN,KG,NaN,NaN,0.0,0.0,0.0,...,0.642536,[786],[],[],[],[786],107594.0,0,0,NaN
1255,888006730,NaN,NaN,NaN,KG,NaN,NaN,0.0,0.0,0.0,...,0.428942,[782],[],[],[],[],122679.0,0,0,NaN


In [8]:
menus = {row.studentno: [row.program_id] for _, row in df.loc[~df.program_id.isna()].iterrows()}

In [9]:
import json

with open("/Users/katherinementzer/SFUSD/sibling_menus.json", "w") as f:
    json.dump(menus, f)

In [9]:
progs = pd.read_csv("~/SFUSD/Data/Cleaned/programs_1819.csv")

In [10]:
print(sum(assigned.merge(progs, on="program_id").studentno) - sum(assigned.studentno))
assigned = assigned.merge(progs, on="program_id")
assigned["remaining_cap"] = assigned.capacity - assigned.studentno

0


In [11]:
sum(assigned.studentno), len(df)

(1290, 1301)

In [12]:
assigned.loc[assigned.remaining_cap == 0]

,program_id,studentno,Unnamed: 0,school_id,program_type,capacity,r2_capacity,r3_capacity,r1_assigned,r1_noenroll,r1_first_choice,max_seats,remaining_cap
87,746-ME-KG,9,102,746,ME,9,9.0,9.0,9.0,0.0,43.0,9.0,0


In [14]:
# rejected = df.loc[df["sibling"] == 729].sort_values("randomnumber", ascending=False).tail(32-24)

In [15]:
# # print(len(df) - len(rejected.studentno))
# accepted = df.loc[~df.studentno.isin(rejected.studentno)]
# # print(len(accepted))
# accepted.loc[:, "program_id"] = [f"{x}-GE-KG" for x in accepted.sibling]
# accepted = accepted.merge(progs[["program_id"]])
# assert len(accepted) == sum(assigned.studentno) - 32 + 24
# # len(accepted), assigned.studentno.sum()

In [13]:
df.loc[~df.program_id.isna()][["studentno"]].to_csv("klm_raw_inputs/preassigned_siblings.csv", index=False)

In [15]:
remaining_capacity = progs.merge(
    assigned[["program_id", "studentno"]],
    how="left",
    on="program_id",
)
# remaining_capacity = progs.merge(assigned[["program_id", "studentno"]], how="left", on="program_id")
remaining_capacity["remaining_cap"] = (
    remaining_capacity.capacity - remaining_capacity.studentno.fillna(0)
).astype(int)
remaining_capacity = remaining_capacity[["program_id", "remaining_cap"]]

ref = pd.read_csv("klm_raw_inputs/program_idxs.csv")
remaining_capacity = ref.merge(remaining_capacity, how="left", on="program_id")
remaining_capacity.iloc[109:113]

,program_idx,program_id,remaining_cap
109,109,735-GE-KG,35
110,110,746-GE-KG,33
111,111,746-ME-KG,0
112,112,746-MN-KG,9


In [18]:
remaining_capacity.to_csv("klm_raw_inputs/school_caps.csv", index=False)

In [19]:
remaining_capacity.remaining_cap.sum()

3776

In [26]:
remaining_capacity.loc[remaining_capacity.remaining_cap <= 0]

,program_idx,program_id,remaining_cap
111,111,746-ME-KG,0


In [27]:
progs.loc[progs.program_id == "746-ME-KG"]

,Unnamed: 0,program_id,school_id,program_type,capacity,r2_capacity,r3_capacity,r1_assigned,r1_noenroll,r1_first_choice,max_seats
102,102,746-ME-KG,746,ME,9,9.0,9.0,9.0,0.0,43.0,9.0


## Sibling FRL Constraint Contribution

In [21]:
df["frl"] = df.freelunch_prob + df.reducedlunch_prob

In [22]:
frl = df.groupby("program_id").sum()[["frl"]]
frl = ref.merge(frl, how="left", on="program_id").fillna(0)
frl = frl.merge(progs[["program_id", "capacity"]], how="left", on="program_id")

In [23]:
frl.sort_values("program_idx").to_csv("klm_raw_inputs/sibling_frl.csv", index=False)

In [24]:
tmp = progs[["program_id", "capacity"]].merge(frl[["program_idx", "program_id", "frl"]], on="program_id")
tmp["ratio"] = tmp.frl / tmp.capacity
tmp.sort_values("ratio", ascending=False).head(10)

,program_id,capacity,program_idx,frl,ratio
102,746-ME-KG,9,111,3.826538,0.425171
145,862-CB-KG,22,150,8.831681,0.401440
93,729-SE-KG,18,107,6.864872,0.381382
91,729-CB-KG,22,105,7.672524,0.348751
42,562-CE-KG,9,52,2.959682,0.328854
58,614-SB-KG,22,69,7.144207,0.324737
18,872-CB-KG,22,154,6.756420,0.307110
10,618-SE-KG,27,71,8.084371,0.299421
153,876-CN-KG,18,157,5.300690,0.294483
95,723-CB-KG,22,101,6.081208,0.276419


In [25]:
frl.sort_values("program_idx")

,program_idx,program_id,frl,capacity
0,0,413-GE-KG,8.556439,96
1,1,413-SA-KG,0.000000,3
2,2,420-GE-KG,1.799632,48
3,3,420-MS-KG,0.000000,3
4,4,420-SE-KG,4.310671,18
...,...,...,...,...
154,154,872-CB-KG,6.756420,22
155,155,872-GE-KG,1.717936,24
156,156,876-CE-KG,2.874964,15
157,157,876-CN-KG,5.300690,18
